### Configure environment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

### Define configuration for this experiment

In [ ]:
slice_name="l4s-new" + fablib.get_bastion_username()

node_conf = [
 {'name': "tx_L4S",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "tx_legacy",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "router", 'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "delay", 'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "rx_L4S",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "rx_legacy",    'cores': 4, 'ram': 32, 'disk': 20, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}
]
net_conf = [
 {"name": "net-tx", "subnet": "10.0.0.0/24", "nodes": [{"name": "tx_L4S",   "addr": "10.0.0.100"}, {"name": "tx_legacy",   "addr": "10.0.0.101"}, {"name": "delay", "addr": "10.0.0.2"}]},
 {"name": "net-delay-router", "subnet": "10.0.2.0/24", "nodes": [{"name": "delay",   "addr": "10.0.2.2"}, {"name": "router", "addr": "10.0.2.1"}]},
 {"name": "net-rx", "subnet": "10.0.5.0/24", "nodes": [{"name": "router",   "addr": "10.0.5.1"}, {"name": "rx_L4S", "addr": "10.0.5.100"}, {"name": "rx_legacy", "addr": "10.0.5.101"}]}

]
route_conf = [
 {"addr": "10.0.5.0/24", "gw": "10.0.0.2", "nodes": ["tx_L4S", "tx_legacy"]}, 
 {"addr": "10.0.5.0/24", "gw": "10.0.2.1", "nodes": ["delay"]},

 {"addr": "10.0.0.0/24", "gw": "10.0.5.1", "nodes": ["rx_L4S", "rx_legacy"]},
 {"addr": "10.0.0.0/24", "gw": "10.0.2.2", "nodes": ["router"]}

]
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }

### Reserve resources

Now, we are ready to reserve resources!

First, make sure you don’t already have a slice with this name:

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

We will select a random site that has sufficient resources for our experiment:

In [ ]:
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_conf['nic']) ):
        break

fablib.show_site(site_name)

Then we will add hosts and network segments:

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])

In [ ]:
# this cell sets up the network segments
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                 name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

-   While it is being prepared, the “State” of the slice will appear as “Configuring”.
-   When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [ ]:
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

### Extend your slice

If you don’t plan to finish an experiment in one day, you can extend your slice. The following cell extends your reservation for 7 days.

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 7 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

### Configure Experiment

In [ ]:
for node in slice.get_nodes():
    # Download and unzip the kernel package
    node.execute("wget https://github.com/L4STeam/linux/releases/download/testing-build/l4s-testing.zip")
    node.execute("sudo apt install unzip")
    node.execute("unzip l4s-testing.zip")
    
    # Install the kernel packages and update GRUB
    node.execute("sudo dpkg --install debian_build/*")
    node.execute("sudo update-grub")
    node.execute("sudo reboot")

# wait for all nodes to come back up
slice.wait_ssh(progress=True)
for node in slice.get_nodes():
    # check kernel version
    node.execute("hostname; uname -a")

In [ ]:
#configuration for DUALPI2 bottleneck
cmd_dualpi2="""sudo apt-get update
sudo apt -y install git gcc make bison flex libdb-dev libelf-dev pkg-config libbpf-dev libmnl-dev libcap-dev libatm1-dev selinux-utils libselinux1-dev
sudo git clone https://github.com/L4STeam/iproute2.git && cd iproute2
sudo ./configure
sudo make
sudo make install"""
slice.get_node(name="router").execute(cmd_dualpi2)
slice.get_node(name="router").execute("sudo modprobe sch_dualpi2")

### Configure resources

Next, we will configure the resources so they are ready to use.

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo apt -y install %s" % pkg)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
# set up static routes
for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
# turn off segmentation offload on interfaces
for iface in slice.get_interfaces():
    iface_name = iface.get_device_name()
    n = iface.get_node()
    offloads = ["gro", "lro", "gso", "tso"]
    for offload in offloads:
        n.execute("sudo ethtool -K %s %s off" % (iface_name, offload))

### Draw the network topology

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Log into resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the resources in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the resources as follows:

-   in Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each resource. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

### Execute Experiment

In [ ]:
# nodes and instances

tx_L4S_node = slice.get_node(name="tx_L4S")
tx_legacy_node = slice.get_node(name="tx_legacy")
rx_L4S_node = slice.get_node(name="rx_L4S")
rx_legacy_node = slice.get_node(name="rx_legacy")
delay_node = slice.get_node(name="delay")
router_node = slice.get_node(name="router")
# interfaces

tx_L4S_egress_iface  = tx_L4S_node.get_interface(network_name = "net-tx")
tx_legacy_egress_iface  = tx_legacy_node.get_interface(network_name = "net-tx")

delay_ingress_tx_iface  = delay_node.get_interface(network_name = "net-tx")
delay_egress_iface  = delay_node.get_interface(network_name = "net-delay-router")
delay_ingress_tx_name = delay_ingress_tx_iface.get_device_name()
delay_egress_name = delay_egress_iface.get_device_name()

router_ingress_iface  = router_node.get_interface(network_name = "net-delay-router")
router_egress_iface  = router_node.get_interface(network_name = "net-rx")
router_egress_name  = router_egress_iface.get_device_name()


rx_L4S_ingress_iface  = rx_L4S_node.get_interface(network_name = "net-rx")
rx_legacy_ingress_iface  = rx_legacy_node.get_interface(network_name = "net-rx")

In [ ]:
# generate full factorial experiment
import itertools

exp_factors = {
    'n_bdp': [0.5, 2, 5, 10],  # n x bandwidth delay product
    'btl_capacity': [100, 1000], #in Mbps #'btl_capacity': [100, 1000]
    'base_rtt': [5, 10, 50, 100], # in ms #'base_rtt': [5, 10, 50, 100],
    'aqm': ['FIFO', 'single_queue_FQ', 'Codel', 'FQ', 'FQ_Codel', 'DualPI2'],
    'ecn_threshold': [1, 5, 20], # in ms #'ecn_threshold': [1, 5, 20]
    'ecn_fallback': [0, 1],  #fallback algorithm, TCP Prague falls back to classic TCP when it detects single queue classic ECN bottleneck # 0: OFF, 1: ON  #'ecn_fallback': [0, 1]
    'rx_L4S_ecn': [0,1,3],  # 0: noecn, 1: ecn, 3: accecn #'rx_L4S_ecn': [0, 1, 3]
    'rx_legacy_ecn': [0,1],  # 0: noecn, 1: ecn #'rx_legacy_ecn': [0, 1]
    'cc_tx_L4S': ["prague"],
    'cc_tx_legacy': ["cubic", "bbr"],
    'trial': [1,2,3,4,5] #'trial': [1, 2, 3, 4, 5]
}

flow_number_tx_L4S=1 #number of tx_L4S flows
flow_number_tx_legacy=1 #number of tx_legacy flows

factor_names = [k for k in exp_factors]
factor_lists = list(itertools.product(*exp_factors.values()))

exp_lists = []

seen_combinations = set()

# Removing ECN factor from FIFO bottleneck because it does not support ECN
# Removing the cases where ECN Threshold is less than or equal to the buffer size in time, these cases are not meaningful in practice

for factor_l in factor_lists:
    temp_dict = dict(zip(factor_names, factor_l))
    if temp_dict['n_bdp'] * temp_dict['base_rtt'] >= temp_dict['ecn_threshold']:
        if temp_dict['aqm'] == 'FIFO':
            del temp_dict['ecn_threshold']
        # Convert dict to a frozenset for set operations
        fs = frozenset(temp_dict.items())
    
        if fs not in seen_combinations:
            seen_combinations.add(fs)
            exp_lists.append(temp_dict)

data_dir_tx_L4S = slice_name + 'singlebottleneck'+"-tx_L4S"
data_dir_tx_legacy = slice_name + 'singlebottleneck'+"-tx_legacy"

print("Number of experiments:",len(exp_lists))

In [ ]:
# run experiments
import time
d = 60 #duration in seconds

em = [delay_ingress_tx_name, delay_egress_name]

for exp in exp_lists:

    # check if we already ran this experiment
    # (allow stop/resume)
    name_tx_L4S="%s_%0.1f_%d_%d_%s_%s_%d_%d_%d_%d" % (exp['cc_tx_L4S'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx_L4S_ecn'], exp['rx_legacy_ecn'], exp['trial'])
    name_tx_legacy="%s_%0.1f_%d_%d_%s_%s_%d_%d_%d_%d" % (exp['cc_tx_legacy'],exp['n_bdp'], exp['btl_capacity'], exp['base_rtt'], exp['aqm'], str(exp.get('ecn_threshold', 'none')), exp['ecn_fallback'], exp['rx_L4S_ecn'], exp['rx_legacy_ecn'], exp['trial'])
    
    file_out_tx_L4S_json = name_tx_L4S+"-result.json"
    stdout_tx_L4S_json, stderr_tx_L4S_json = tx_L4S_node.execute("ls " + file_out_tx_L4S_json, quiet=True) 
    
    file_out_tx_legacy_json =name_tx_legacy+"-result.json"
    stdout_tx_legacy_json, stderr_tx_legacy_json = tx_legacy_node.execute("ls " + file_out_tx_legacy_json, quiet=True) 
    

    if len(stdout_tx_L4S_json) and len(stdout_tx_legacy_json):
        print("Already have " + name_tx_L4S + " and "+ name_tx_legacy + ", skipping")

    elif len(stderr_tx_L4S_json) or len(stderr_tx_legacy_json):
        print("Running experiment to generate " + name_tx_L4S + " and "+ name_tx_legacy) 
        
        
        tx_L4S_node.execute("sudo modprobe tcp_" + exp['cc_tx_L4S'])
        tx_L4S_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + exp['cc_tx_L4S'])
        tx_L4S_node.execute("sudo sysctl -w net.ipv4.tcp_ecn=3") # it is assumed that the L4S sender uses AccECN
        
        tx_legacy_node.execute("sudo modprobe tcp_" + exp['cc_tx_legacy'])
        tx_legacy_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + exp['cc_tx_legacy'])
        tx_legacy_node.execute("sudo sysctl -w net.ipv4.tcp_ecn=1") # it is assumed that the legacy sender uses Classic ECN
        
        # delay at emulator
        for e in em:
            cmds = "sudo tc qdisc replace dev {iface} root netem delay {owd}ms limit 60000".format(iface=e, owd=exp['base_rtt']/2)
            delay_node.execute(cmds)
        
        # fixed values
        btl_limit    = int(1000*exp['n_bdp']*exp['btl_capacity']*exp['base_rtt']/8) # limit of the bottleneck, n_bdp x BDP in bytes 
        packet_number=int(btl_limit/1500)+1
        
        
        #ecn-fallback configuration
               
        commands = "echo {value} | sudo tee /sys/module/tcp_prague/parameters/prague_ecn_fallback".format(value=str(exp['ecn_fallback']))
        tx_L4S_node.execute(commands)
        
        #receiver ecn configuration
                
        rx_L4S_node.execute("sudo sysctl -w net.ipv4.tcp_ecn={ecn_type}".format(ecn_type=exp['rx_L4S_ecn']))
        rx_legacy_node.execute("sudo sysctl -w net.ipv4.tcp_ecn={ecn_type}".format(ecn_type=exp['rx_legacy_ecn']))
        
        #aqm type selection
        cmds_prefix = '''
            sudo tc qdisc del dev {iface} root
            sudo tc qdisc replace dev {iface} root handle 1: htb default 3 
            sudo tc class add dev {iface} parent 1: classid 1:3 htb rate {capacity}mbit 
            '''.format(iface=router_egress_name, capacity=exp['btl_capacity'], buffer=btl_limit)
        
        cmds_specific_initial = "sudo tc qdisc replace dev {iface} parent 1:3 handle 3: ".format(iface=router_egress_name)
        
        cmds_specific = {
        'FIFO': "bfifo limit {buffer}".format(buffer=btl_limit),
        'single_queue_FQ': "fq limit {packet_limit} flow_limit {packet_limit} orphan_mask 0 ce_threshold {threshold}ms".format(packet_limit=packet_number, threshold=exp.get('ecn_threshold', 0)),
        'Codel': "codel limit {packet_limit} target {target}ms interval 100ms ecn ce_threshold {threshold}ms".format(packet_limit=packet_number, target=exp['base_rtt']*exp['n_bdp'], threshold=exp.get('ecn_threshold', 0)),
        'FQ': "fq limit {packet_limit} flow_limit {packet_limit} ce_threshold {threshold}ms".format(packet_limit=packet_number, threshold=exp.get('ecn_threshold', 0)),
        'FQ_Codel': "fq_codel limit {packet_limit} target {target}ms interval 100ms ecn ce_threshold {threshold}ms".format(packet_limit=packet_number, target=exp['base_rtt']*exp['n_bdp'], threshold=exp.get('ecn_threshold', 0)),
        'DualPI2': "dualpi2 target {threshold}ms".format(threshold=exp.get('ecn_threshold', 0))
        }

        cmds_aqm = {key: cmds_specific_initial + cmd for key, cmd in cmds_specific.items()}
        
        router_node.execute(cmds_prefix)
        router_node.execute(cmds_aqm[ exp['aqm'] ])
            
        rx_L4S_node.execute("killall iperf3")
        rx_legacy_node.execute("killall iperf3")
        

        #print("Starting experiment with {1} bdp {2} capacity {3} rtt {4} {5} thrshold {6} ecn_fallback {7} rx_L4S {8} rx_legacy for {duration} seconds".format(duration=d, 1=exp['n_bdp'], 2=exp['btl_capacity'], 3=exp['base_rtt'], 4=exp['aqm'], 5=exp['ecn_threshold'], 6= exp['ecn_fallback'], 7=exp['rx_L4S_ecn'], 8=exp['rx_legacy_ecn']))
        
        rx_L4S_node.execute("iperf3 -s -1 -p 4000 -D")
        rx_legacy_node.execute("iperf3 -s -1 -p 5000 -D")
        
        tx_L4S_node.execute_thread("sleep 1; iperf3 -c 10.0.5.100 -t {duration} -P {flows} -C {cc} -p 4000 -J > {flow}-result.json".format(flow =name_tx_L4S, duration=d, flows=flow_number_tx_L4S, cc=exp['cc_tx_L4S']))
        stdout, stderr = tx_legacy_node.execute("sleep 1; iperf3 -c 10.0.5.101 -t {duration} -P {flows} -C {cc} -p 5000 -J > {flow}-result.json".format(flow =name_tx_legacy, duration=d, flows=flow_number_tx_legacy, cc=exp['cc_tx_legacy']))
        time.sleep(3)  # time.sleep(1)
        
print("finished")

### Analysis of the results

In [ ]:
tx_L4S_node.execute('mkdir '+data_dir_tx_L4S)

tx_L4S_node.execute('mv *.json '+ data_dir_tx_L4S)
tx_L4S_node.execute('tar -czvf '+data_dir_tx_L4S+ '.tgz ' +  data_dir_tx_L4S)


tx_legacy_node.execute('mkdir '+data_dir_tx_legacy)

tx_legacy_node.execute('mv *.json '+ data_dir_tx_legacy)      
tx_legacy_node.execute('tar -czvf '+data_dir_tx_legacy+ '.tgz ' +  data_dir_tx_legacy)

In [ ]:
tx_L4S_node.upload_file("/home/fabric/work/analysis.py", f"/home/ubuntu/{data_dir_tx_L4S}/analysis.py")
tx_legacy_node.upload_file("/home/fabric/work/analysis.py", f"/home/ubuntu/{data_dir_tx_legacy}/analysis.py")

In [ ]:
tx_L4S_node.execute(f'python3 /home/ubuntu/{data_dir_tx_L4S}/analysis.py')
tx_legacy_node.execute(f'python3 /home/ubuntu/{data_dir_tx_legacy}/analysis.py')

In [ ]:
tx_L4S_node.download_file("/home/fabric/work/tput_tx_L4S.json",f"/home/ubuntu/{data_dir_tx_L4S}/throughput_data.json")
tx_L4S_node.download_file("/home/fabric/work/srtt_tx_L4S.json",f"/home/ubuntu/{data_dir_tx_L4S}/srtt_data.json")

tx_legacy_node.download_file("/home/fabric/work/tput_tx_legacy.json",f"/home/ubuntu/{data_dir_tx_legacy}/throughput_data.json")
tx_legacy_node.download_file("/home/fabric/work/srtt_tx_legacy.json",f"/home/ubuntu/{data_dir_tx_legacy}/srtt_data.json")

In [ ]:
import json
import os

# Initialize empty variables
throughput_data = {}
srtt_data = {}

# Directory containing JSON files
data_directory = '/home/fabric/work/'

# List of JSON files in the directory
json_files = [f for f in os.listdir(data_directory) if f.endswith('.json')]

# Load data from each JSON file and update the variables
for file_name in json_files:
    file_path = os.path.join(data_directory, file_name)
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Check if the file contains throughput data or srtt data based on its name
    if 'tput' in file_name:
        throughput_data.update(data)
    elif 'srtt' in file_name:
        srtt_data.update(data)

In [ ]:
throughput_data

In [ ]:
srtt_data

In [ ]:

def plot_heatmap_for_fixed_btl(sorted_data_tx_L4S, sorted_data_tx_legacy):

    # Initialize an empty dictionary to hold the heatmap data
    heatmap_data = {}

    # Loop through each rtt value under the fixed bottleneck
    for yval in sorted_data_tx_L4S.keys():
        # Loop through each AQM type
        for aqm in sorted_data_tx_L4S[yval].keys():
            prague_throughput = sorted_data_tx_L4S[yval][aqm]
            cubic_throughput = sorted_data_tx_legacy[yval][aqm]
            share = prague_throughput / (prague_throughput + cubic_throughput)

            if yval not in heatmap_data:
                heatmap_data[yval] = {}

            heatmap_data[yval][aqm] = share

    return heatmap_data

def plot_heatmap_for_rtt(sorted_data_tx_L4S, sorted_data_tx_legacy):
    # Initialize an empty dictionary to hold the heatmap data
    heatmap_data = {}

    # Loop through each rtt value under the fixed bottleneck
    for yval in sorted_data_tx_L4S.keys():
        # Loop through each AQM type
        for aqm in sorted_data_tx_L4S[yval].keys():
            prague_rtt = sorted_data_tx_L4S[yval][aqm]
            cubic_rtt = sorted_data_tx_legacy[yval][aqm]
            cubic_relative_diff = (cubic_rtt - prague_rtt) / cubic_rtt

            if yval not in heatmap_data:
                heatmap_data[yval] = {}

            heatmap_data[yval][aqm] = cubic_relative_diff

    return heatmap_data

!pip install seaborn

In [ ]:

import shutil, tarfile

import json

import numpy as np

import matplotlib.pyplot as plt

import pandas as pd

import re

from statistics import mean

import itertools

import seaborn as sns

import os
from itertools import product
from matplotlib.colors import ListedColormap

specified_params1 = {
    'btl_capacity': [100],
    #'n_bdp': [0.5, 2, 5, 10],
    'base_rtt': [5, 10, 50], 
    'ecn_threshold': [1, 5],
    'ecn_fallback': [0], 
    'rx_L4S_ecn': [2],
    'rx_legacy_ecn': [1],
    # 'aqm': 'FIFO'

}

keys = specified_params1.keys() 
values = (specified_params1[key] for key in keys)
combinations = [dict(zip(keys, combination)) for combination in product(*values)]

exp_factors['btl_capacity'] 
exp_factors['base_rtt']  

factor_x = 'aqm'  # choose which parameter you want to observe
factor_y = 'n_bdp'  # choose which parameter you want to observe

# Create nested dictionaries to store data by btl_capacity and base_rtt

fig1, axes1 = plt.subplots(2, 3, figsize=(53, 30))
fig2, axes2 = plt.subplots(2, 3, figsize=(53, 30)) # Adjust the figsize as needed

for index, i in enumerate(combinations):
    specified_params=i
    relevant_data_tx_L4S = {}
    relevant_data_tx_legacy = {}
    relevant_srtt_data_tx_L4S = {}
    relevant_srtt_data_tx_legacy = {}
    
    for exp in exp_lists:
        is_relevant = all(
            (k == 'ecn_threshold' and (v == exp.get(k) or exp.get(k) is None)) or
            (k != 'ecn_threshold' and v == exp.get(k)) for k, v in specified_params.items()
        )
    
        if is_relevant:
            btl = exp['btl_capacity']
            rtt = exp['base_rtt']
    
            name_tx_L4S = "%s_%0.1f_%d_%d_%s_%s_%d_%d_%d_%d" % (
                exp['cc_tx_L4S'], exp['n_bdp'], btl, rtt, exp['aqm'], str(exp.get('ecn_threshold', 'none')),
                exp['ecn_fallback'],
                exp['rx_L4S_ecn'], exp['rx_legacy_ecn'], exp['trial'])
            name_tx_legacy = "%s_%0.1f_%d_%d_%s_%s_%d_%d_%d_%d" % (
                exp['cc_tx_legacy'], exp['n_bdp'], btl, rtt, exp['aqm'], str(exp.get('ecn_threshold', 'none')),
                exp['ecn_fallback'],
                exp['rx_L4S_ecn'], exp['rx_legacy_ecn'], exp['trial'])
    
            xval = exp[factor_x]
            yval= exp[factor_y]
            # Update relevant_data_tx_L4S
            if name_tx_L4S in throughput_data:
                if yval not in relevant_data_tx_L4S:
                    relevant_data_tx_L4S[yval] = {}
                if xval not in relevant_data_tx_L4S[yval]:
                    relevant_data_tx_L4S[yval][xval] = []
                relevant_data_tx_L4S[yval][xval].append(throughput_data[name_tx_L4S])
    
            # Update relevant_data_tx_legacy
            if name_tx_legacy in throughput_data:
                if yval not in relevant_data_tx_legacy:
                    relevant_data_tx_legacy[yval] = {}
                if xval not in relevant_data_tx_legacy[yval]:
                    relevant_data_tx_legacy[yval][xval] = []
                relevant_data_tx_legacy[yval][xval].append(throughput_data[name_tx_legacy])
    
            # Update relevant_srtt_data_tx_L4S
            if name_tx_L4S in srtt_data:
                if yval not in relevant_srtt_data_tx_L4S:
                    relevant_srtt_data_tx_L4S[yval] = {}
                if xval not in relevant_srtt_data_tx_L4S[yval]:
                    relevant_srtt_data_tx_L4S[yval][xval] = []
                relevant_srtt_data_tx_L4S[yval][xval].append(srtt_data[name_tx_L4S]-rtt)
    
            # Update relevant_srtt_data_tx_legacy
            if name_tx_legacy in srtt_data:
                if yval not in relevant_srtt_data_tx_legacy:
                    relevant_srtt_data_tx_legacy[yval] = {}
                if xval not in relevant_srtt_data_tx_legacy[yval]:
                    relevant_srtt_data_tx_legacy[yval][xval] = []
                relevant_srtt_data_tx_legacy[yval][xval].append(srtt_data[name_tx_legacy]-rtt)

    # Average the throughputs for tx_L4S
    for yval, xval_data in relevant_data_tx_L4S.items():
        for xval, throughputs in xval_data.items():
            relevant_data_tx_L4S[yval][xval] = np.mean(throughputs)
    
    # Average the throughputs for tx_legacy
    for yval, xval_data in relevant_data_tx_legacy.items():
        for xval, throughputs in xval_data.items():
            relevant_data_tx_legacy[yval][xval] = np.mean(throughputs)
    
    # Average the srtts for tx_L4S
    for yval, xval_data in relevant_srtt_data_tx_L4S.items():
        for xval, srtts in xval_data.items():
            relevant_srtt_data_tx_L4S[yval][xval] = np.mean(srtts) 
    
    # Average the srtts for tx_legacy
    for yval, xval_data in relevant_srtt_data_tx_legacy.items():
        for xval, srtts in xval_data.items():
            relevant_srtt_data_tx_legacy[yval][xval] = np.mean(srtts)
    
    # Sort and get values for each btl and rtt combination
    sorted_throughputs_tx_L4S = {}
    sorted_throughputs_tx_legacy = {}
    sorted_rtts_tx_L4S = {}
    sorted_rtts_tx_legacy = {}
    
    for yval in sorted(relevant_data_tx_L4S.keys()):
        sorted_throughputs_tx_L4S[yval] = {}
        sorted_rtts_tx_L4S[yval] = {}
        
        for xval in sorted(relevant_data_tx_L4S[yval].keys()):
            sorted_throughputs_tx_L4S[yval][xval] = relevant_data_tx_L4S[yval].get(xval, None)
            sorted_rtts_tx_L4S[yval][xval] = relevant_srtt_data_tx_L4S[yval].get(xval, None)
    
    
    for yval in sorted(relevant_data_tx_legacy.keys()):
        sorted_throughputs_tx_legacy[yval] = {}
        sorted_rtts_tx_legacy[yval] = {}
        
        for xval in sorted(relevant_data_tx_legacy[yval].keys()):
            sorted_throughputs_tx_legacy[yval][xval] = relevant_data_tx_legacy[yval].get(xval, None)
            sorted_rtts_tx_legacy[yval][xval] = relevant_srtt_data_tx_legacy[yval].get(xval, None)
    
    
    title_map = {
    'btl_capacity': lambda: f"{specified_params['btl_capacity']}Mbps-",
    'n_bdp': lambda: f"{specified_params['n_bdp']}BDP-",
    'base_rtt': lambda: f"{specified_params['base_rtt']}ms RTT-",
    'ecn_threshold': lambda: f"ECN Threshold = {specified_params['ecn_threshold']}ms -",
    'rx_L4S_ecn': lambda: f"ServerA ECN={specified_params['rx_L4S_ecn']}-",
    'rx_legacy_ecn': lambda: f"ServerB ECN={specified_params['rx_legacy_ecn']}-",
    'ecn_fallback': lambda: f"ECN Fallback={specified_params['ecn_fallback']}"
     }
    

    
    desired_order = ['FIFO', 'single_queue_FQ', 'Codel', 'FQ', 'FQ_Codel', 'DualPI2']
    #desired_order = ['FIFO', 'single_queue_FQ']
    
    # Convert the dictionary to a Pandas DataFrame
    df = pd.DataFrame(plot_heatmap_for_fixed_btl(sorted_throughputs_tx_L4S, sorted_throughputs_tx_legacy))
    df = df.reindex(desired_order)
    df=df.rename(index={'FQ_Codel': 'FQ Codel'})
    df=df.rename(index={'single_queue_FQ': 'Single  \nQueue FQ'})
    df_rtt = pd.DataFrame(plot_heatmap_for_rtt(sorted_rtts_tx_L4S, sorted_rtts_tx_legacy))
    df_rtt = df_rtt.reindex(desired_order)
    df_rtt=df_rtt.rename(index={'FQ_Codel': 'FQ Codel'})
    df_rtt=df_rtt.rename(index={'single_queue_FQ': 'Single  \nQueue FQ'})
    df_masked = np.ma.masked_invalid(df)
    df_masked_rtt = np.ma.masked_invalid(df_rtt)

    
    dynamic_title = ''.join(val() for key, val in title_map.items() if key != factor_y)
    cmap = plt.cm.coolwarm  # Start with the coolwarm colormap
    cmap.set_bad(color='black')  # Set the color for masked values (None/NaN)
    
    #Plot the heatmap
    #plt.figure(figsize=(12, 8))
    
    ax1 = axes1[index % 2, index // 2] 
    #ax1 = axes1[index]
    sns.heatmap(df, annot=True, cmap=cmap, cbar_kws={'label': 'Prague Throughput Share'}, annot_kws={"size": 50}, vmin=0, vmax=1, ax=ax1, cbar=False)
    ax1.set_title(dynamic_title, fontsize=20)
    ax1.set_xlabel('Buffer Size (n x BDP)', fontsize=20)
    ax1.set_ylabel("AQM Types", fontsize=20)
    ax1.set_yticklabels(ax1.get_yticklabels(), fontsize=10) # Adjust for y-axis labels
    ax1.set_xticklabels(ax1.get_xticklabels(), fontsize=20) # Adjust for x-axis labels
    
    ax2 = axes2[index % 2, index // 2] 
    #ax2 = axes2[index] 
    sns.heatmap(df_rtt, annot=True, cmap=cmap, cbar_kws={'label': 'Cubic Relative Queuing Delay'}, annot_kws={"size": 50},  vmin=-1, vmax=1, ax=ax2, cbar=False)
    ax2.set_title(dynamic_title, fontsize=20)
    #ax2.set_xlabel(factor_y, fontsize=40)
    ax2.set_xlabel('Buffer Size (n x BDP)', fontsize=20)
    ax2.set_ylabel("AQM Types", fontsize=20)
    ax2.set_yticklabels(ax2.get_yticklabels(), fontsize=20) # Adjust for y-axis labels
    ax2.set_xticklabels(ax2.get_xticklabels(), fontsize=20) # Adjust for x-axis labels
    
    
plt.figure(fig1.number)
fig1.suptitle('100 Mbps - Server A ECN: AccECN - Server B ECN: Classic ECN - ECN Fallback: OFF', fontsize=45) 
#fig1.subplots_adjust(hspace=0.15)  # Adjust the vertical spacing
cbar_ax = fig1.add_axes([0.92, 0.15, 0.02, 0.7])  # x-position, y-position, width, height
cbar=fig1.colorbar(ax1.collections[0], cax=cbar_ax)
cbar.set_label('Prague Throughput Share', size=40)  
cbar.ax.tick_params(labelsize=35)
plt.tight_layout(rect=[0, 0.01, 0.9, 0.98], pad=1.08, h_pad=1.5, w_pad=1.08)
plt.savefig('/home/fabric/work/throughput.png', dpi=100)



plt.figure(fig2.number)
fig2.suptitle('100 Mbps - Server A ECN: AccECN - Server B ECN: Classic ECN - ECN Fallback: OFF', fontsize=45) 
cbar_ax = fig2.add_axes([0.92, 0.15, 0.02, 0.7])  # x-position, y-position, width, height
cbar=fig2.colorbar(ax2.collections[0], cax=cbar_ax)
cbar.set_label('Cubic Relative Queuing Delay', size=40)  
cbar.ax.tick_params(labelsize=35)
plt.tight_layout(rect=[0, 0.01, 0.9, 0.98], pad=1.08, h_pad=1.5, w_pad=1.08)
plt.savefig('/home/fabric/work/relativedelay.png', dpi=100)
 